# author identification

In [1]:
import pandas as pd
import matplotlib.pyplot as mltp
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
import sklearn as sk
import seaborn as sea
import re 
from tqdm import tqdm
from statistics import median


Durchführung:
1. Cleaning & Preprocessing
    1. Removing punctuation
    2. Lemmatization
    3. Sonderzeichen, welche unnutzbar sind
2. tokenize
    1. word_tokenize
    2. sent_tokenize
    3. stopwords
    4. stemwords


Zu hohe Anzahl an Daten, weshalb eine Reduktion dieser wichtig wurde, von fast 700k zu 20k und 6k Testdaten
1. Nur Englischsprachige
2. Haben 39 Kategorien --> je 500 Einträge, auf welchen trainiert wird

Nächste Schritte:
- LDA machen für die Topics und deren Wörter
- ML basierend auf den Vektoren machen, um die Eigenschaften zu identifizieren
- anbinden an Frontend für eine Angabe der Eigenschaften

In [2]:
def stripping(liste):
    return [i.strip() for i in liste]

In [3]:
df=pd.read_csv('data/df_tokenized_port_2.csv',converters={'sent_tokenize': lambda x: x[1:-1].strip('][').replace("'","").strip().split(','),'word_tokenize': lambda x: x[1:-1].strip('][').replace("'","").strip().split(','),
'word_tokenize_without_stopwords': lambda x: x[1:-1].strip('][').replace("'","").strip().split(',')
,'word_tokenize_without_stopwords_port': lambda x: x[1:-1].strip('][').replace("'","").strip().split(',')
})



In [4]:
df['word_tokenize']=df['word_tokenize'].apply(stripping)
df['sent_tokenize']=df['sent_tokenize'].apply(stripping)
df['word_tokenize_without_stopwords']=df['word_tokenize_without_stopwords'].apply(stripping)
df['word_tokenize_without_stopwords_port']=df['word_tokenize_without_stopwords_port'].apply(stripping)

In [5]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]


In [7]:
len(df.index)

644247

In [6]:
df.head()

,id,gender,age,topic,sign,date,text,lang,language_2,word_tokenize,sent_tokenize,count_word,count_sent,word_tokenize_num_of_stopwords,word_tokenize_without_stopwords,word_tokenize_without_stopwords_port
0,2059027,male,15,Student,Leo,"14,May,2004",Info has been found (+/- 100 pages and 4.5 MB ...,en,en,"[Info, has, been, found, (, +/-, 100, pages, a...",[Info has been found (+/- 100 pages and 4.5 MB...,31,1,11,"[Info, found, (, +/-, 100, pages, 4.5, MB, .pd...","[info, found, (, +/-, 100, page, 4.5, mb, .pdf..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewes van der Laa...,en,en,"[These, are, the, team, members, :, Drewes, va...",[These are the team members: Drewes van der La...,23,1,3,"[These, team, members, :, Drewes, van, der, La...","[these, team, member, :, drew, van, der, laag,..."
2,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo's Toolbar I can now 'capture' ...,en,en,"[Thanks, to, Yahoo, ""s"", Toolbar, I, can, now,...","[""Thanks to Yahoos Toolbar I can now capture t...",77,3,26,"[Thanks, Yahoo, ""s"", Toolbar, I, ""capture"", """"...","[thank, yahoo, ""s"", toolbar, i, ""captur"", """", ..."
3,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation with my Dad ...,en,en,"[I, had, an, interesting, conversation, with, ...",[I had an interesting conversation with my Dad...,760,36,298,"[I, interesting, conversation, Dad, morning, ....","[i, interest, convers, dad, morn, ., we, talk,..."
4,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of summing up thin...,en,en,"[Somehow, Coca-Cola, has, a, way, of, summing,...",[Somehow Coca-Cola has a way of summing up thi...,226,14,91,"[Somehow, Coca-Cola, way, summing, things, wel...","[somehow, coca-cola, way, sum, thing, well, .,..."


In [49]:
df[(df['count_sent']>1)&(df['count_sent']<20)&(df['count_word']>1)&(df['count_word']<200)]

,id,gender,age,topic,sign,date,text,lang,language_2,word_tokenize,sent_tokenize,count_word,count_sent,word_tokenize_num_of_stopwords,word_tokenize_without_stopwords,word_tokenize_without_stopwords_port
2,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo's Toolbar I can now 'capture' ...,en,en,"[Thanks, to, Yahoo, ""s"", Toolbar, I, can, now,...","[""Thanks to Yahoos Toolbar I can now capture t...",77,3,26,"[Thanks, Yahoo, ""s"", Toolbar, I, ""capture"", """"...","[thank, yahoo, ""s"", toolbar, i, ""captur"", """", ..."
7,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a lot looking fo...,en,en,"[I, surf, the, English, news, sites, a, lot, l...","[""I surf the English news sites a lot looking ...",183,7,64,"[I, surf, English, news, sites, lot, looking, ...","[i, surf, english, news, site, lot, look, tidb..."
10,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",There is so much that is different here from a...,en,en,"[There, is, so, much, that, is, different, her...","[""There is so much that is different here from...",74,4,26,"[There, much, different, anything, I, ""ve"", ev...","[there, much, differ, anyth, i, ""ve"", ever, se..."
19,3581210,male,33,InvestmentBanking,Aquarius,"22,June,2004",You may have noticed a new feature on my blog....,en,en,"[You, may, have, noticed, a, new, feature, on,...","[""You may have noticed a new feature on my blo...",179,7,65,"[You, may, noticed, new, feature, blog, ..., ""...","[you, may, notic, new, featur, blog, ..., ""s"",..."
21,3581210,male,33,InvestmentBanking,Aquarius,"02,July,2004",It seems everything is not all that smooth in ...,en,en,"[It, seems, everything, is, not, all, that, sm...",[It seems everything is not all that smooth in...,92,4,39,"[It, seems, everything, smooth, Seoul, concern...","[it, seem, everyth, smooth, seoul, concern, re..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644240,1713845,male,23,Student,Taurus,"02,July,2004",Dear Susan There once was a bird the size of a...,en,en,"[Dear, Susan, There, once, was, a, bird, the, ...",[Dear Susan There once was a bird the size of ...,64,5,26,"[Dear, Susan, There, bird, size, turd, sitting...","[dear, susan, there, bird, size, turd, sit, te..."
644241,1713845,male,23,Student,Taurus,"01,July,2004",Dear Susan you keep asking me who your 'dad' i...,en,en,"[Dear, Susan, you, keep, asking, me, who, your...","[""Dear Susan you keep asking me who your dad i...",101,8,49,"[Dear, Susan, keep, asking, ""dad"", """", ., gues...","[dear, susan, keep, ask, ""dad"", """", ., guess, ..."
644242,1713845,male,23,Student,Taurus,"01,July,2004",Dear Susan I could write some really bitter di...,en,en,"[Dear, Susan, I, could, write, some, really, b...",[Dear Susan I could write some really bitter d...,45,4,15,"[Dear, Susan, I, could, write, really, bitter,...","[dear, susan, i, could, write, realli, bitter,..."
644243,1713845,male,23,Student,Taurus,"01,July,2004",Dear Susan 'I have the second yeast infection ...,en,en,"[Dear, Susan, """", I, have, the, second, yeast,...","[""Dear Susan I have the second yeast infection...",77,5,24,"[Dear, Susan, """", I, second, yeast, infection,...","[dear, susan, """", i, second, yeast, infect, pa..."


In [50]:
df['word_tokenize_without_stopwords'][0]

['Info',
 'found',
 '(',
 '+/-',
 '100',
 'pages',
 '4.5',
 'MB',
 '.pdf',
 'files',
 ')',
 'Now',
 'wait',
 'untill',
 'team',
 'leader',
 'processed',
 'learns',
 'html',
 '.']

In [51]:
len(df.index)


644247

In [52]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/niclascramer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/niclascramer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [53]:
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
df['msg_lemmatized']=df['word_tokenize_without_stopwords'].apply(lambda x:lemmatizer(x))

In [54]:
df.head(5)

,id,gender,age,topic,sign,date,text,lang,language_2,word_tokenize,sent_tokenize,count_word,count_sent,word_tokenize_num_of_stopwords,word_tokenize_without_stopwords,word_tokenize_without_stopwords_port,msg_lemmatized
0,2059027,male,15,Student,Leo,"14,May,2004",Info has been found (+/- 100 pages and 4.5 MB ...,en,en,"[Info, has, been, found, (, +/-, 100, pages, a...",[Info has been found (+/- 100 pages and 4.5 MB...,31,1,11,"[Info, found, (, +/-, 100, pages, 4.5, MB, .pd...","[info, found, (, +/-, 100, page, 4.5, mb, .pdf...","[Info, found, (, +/-, 100, page, 4.5, MB, .pdf..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewes van der Laa...,en,en,"[These, are, the, team, members, :, Drewes, va...",[These are the team members: Drewes van der La...,23,1,3,"[These, team, members, :, Drewes, van, der, La...","[these, team, member, :, drew, van, der, laag,...","[These, team, member, :, Drewes, van, der, Laa..."
2,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo's Toolbar I can now 'capture' ...,en,en,"[Thanks, to, Yahoo, ""s"", Toolbar, I, can, now,...","[""Thanks to Yahoos Toolbar I can now capture t...",77,3,26,"[Thanks, Yahoo, ""s"", Toolbar, I, ""capture"", """"...","[thank, yahoo, ""s"", toolbar, i, ""captur"", """", ...","[Thanks, Yahoo, ""s"", Toolbar, I, ""capture"", """"..."
3,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation with my Dad ...,en,en,"[I, had, an, interesting, conversation, with, ...",[I had an interesting conversation with my Dad...,760,36,298,"[I, interesting, conversation, Dad, morning, ....","[i, interest, convers, dad, morn, ., we, talk,...","[I, interesting, conversation, Dad, morning, ...."
4,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of summing up thin...,en,en,"[Somehow, Coca-Cola, has, a, way, of, summing,...",[Somehow Coca-Cola has a way of summing up thi...,226,14,91,"[Somehow, Coca-Cola, way, summing, things, wel...","[somehow, coca-cola, way, sum, thing, well, .,...","[Somehow, Coca-Cola, way, summing, thing, well..."


In [55]:
import re
df['cleanLinks'] = df['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [56]:
df=df[~df['text'].str.contains('https:\/\/.*')]

In [79]:
def laenge_der_Saetze(liste):
    laenge=[]
    for x in liste:
        laenge.append(len(x.split()))
    return median(laenge)

In [73]:
def läenge(df):
    return len(df['word_tokenize'])/len(df['sent_tokenize'])

In [86]:
df['word_per_sent_mean']=round(df['count_word']/df['count_sent'],2)

In [87]:
df.head(5)

,id,gender,age,topic,sign,date,text,lang,language_2,word_tokenize,sent_tokenize,count_word,count_sent,word_tokenize_num_of_stopwords,word_tokenize_without_stopwords,word_tokenize_without_stopwords_port,msg_lemmatized,cleanLinks,laenge_saetze,word_per_sent_mean
0,2059027,male,15,Student,Leo,"14,May,2004",Info has been found (+/- 100 pages and 4.5 MB ...,en,en,"[Info, has, been, found, (, +/-, 100, pages, a...",[Info has been found (+/- 100 pages and 4.5 MB...,31,1,11,"[Info, found, (, +/-, 100, pages, 4.5, MB, .pd...","[info, found, (, +/-, 100, page, 4.5, mb, .pdf...","[Info, found, (, +/-, 100, page, 4.5, MB, .pdf...",Info has been found (+/- 100 pages and 4.5 MB ...,1,31.00
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewes van der Laa...,en,en,"[These, are, the, team, members, :, Drewes, va...",[These are the team members: Drewes van der La...,23,1,3,"[These, team, members, :, Drewes, van, der, La...","[these, team, member, :, drew, van, der, laag,...","[These, team, member, :, Drewes, van, der, Laa...",These are the team members: Drewes van der Laa...,1,23.00
2,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo's Toolbar I can now 'capture' ...,en,en,"[Thanks, to, Yahoo, ""s"", Toolbar, I, can, now,...","[""Thanks to Yahoos Toolbar I can now capture t...",77,3,26,"[Thanks, Yahoo, ""s"", Toolbar, I, ""capture"", """"...","[thank, yahoo, ""s"", toolbar, i, ""captur"", """", ...","[Thanks, Yahoo, ""s"", Toolbar, I, ""capture"", """"...",Thanks to Yahoo's Toolbar I can now 'capture' ...,3,25.67
3,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation with my Dad ...,en,en,"[I, had, an, interesting, conversation, with, ...",[I had an interesting conversation with my Dad...,760,36,298,"[I, interesting, conversation, Dad, morning, ....","[i, interest, convers, dad, morn, ., we, talk,...","[I, interesting, conversation, Dad, morning, ....",I had an interesting conversation with my Dad ...,36,21.11
4,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of summing up thin...,en,en,"[Somehow, Coca-Cola, has, a, way, of, summing,...",[Somehow Coca-Cola has a way of summing up thi...,226,14,91,"[Somehow, Coca-Cola, way, summing, things, wel...","[somehow, coca-cola, way, sum, thing, well, .,...","[Somehow, Coca-Cola, way, summing, thing, well...",Somehow Coca-Cola has a way of summing up thin...,14,16.14


Wörter von 1 bis 707
Sätze 1 bis 34

In [94]:
len(df[(df['count_word']<708)&(df['count_sent']<34)].index)

591760

In [95]:
df=df[(df['count_word']<708)&(df['count_sent']<34)]

In [89]:
df.to_csv('data/final_gesamt.csv')

In [92]:
len(df['topic'].unique())

40

In [90]:
df['topic'].value_counts()

indUnk                     238179
Student                    145103
Technology                  39631
Arts                        30754
Education                   28213
Communications-Media        19085
Internet                    15006
Non-Profit                  13906
Engineering                 10873
Law                          8685
Publishing                   7356
Science                      6824
Government                   6459
Consulting                   5516
Religion                     5020
Fashion                      4539
Marketing                    4492
Advertising                  4425
BusinessServices             4194
Banking                      3776
Accounting                   3662
Chemicals                    3596
Telecommunications           3561
Military                     3006
Museums-Libraries            2999
Sports-Recreation            2886
HumanResources               2833
RealEstate                   2783
Transportation               2220
Manufacturing 

In [121]:

df.iloc[[0, 1, 2, 4, 5, 6, 7, 8, 9, 10]]

,id,gender,age,topic,sign,date,text,lang,language_2,word_tokenize,sent_tokenize,count_word,count_sent,word_tokenize_num_of_stopwords,word_tokenize_without_stopwords,word_tokenize_without_stopwords_port,msg_lemmatized,cleanLinks,laenge_saetze,word_per_sent_mean
0,2059027,male,15,Student,Leo,"14,May,2004",Info has been found (+/- 100 pages and 4.5 MB ...,en,en,"[Info, has, been, found, (, +/-, 100, pages, a...",[Info has been found (+/- 100 pages and 4.5 MB...,31,1,11,"[Info, found, (, +/-, 100, pages, 4.5, MB, .pd...","[info, found, (, +/-, 100, page, 4.5, mb, .pdf...","[Info, found, (, +/-, 100, page, 4.5, MB, .pdf...",Info has been found (+/- 100 pages and 4.5 MB ...,1,31.00
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewes van der Laa...,en,en,"[These, are, the, team, members, :, Drewes, va...",[These are the team members: Drewes van der La...,23,1,3,"[These, team, members, :, Drewes, van, der, La...","[these, team, member, :, drew, van, der, laag,...","[These, team, member, :, Drewes, van, der, Laa...",These are the team members: Drewes van der Laa...,1,23.00
2,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo's Toolbar I can now 'capture' ...,en,en,"[Thanks, to, Yahoo, ""s"", Toolbar, I, can, now,...","[""Thanks to Yahoos Toolbar I can now capture t...",77,3,26,"[Thanks, Yahoo, ""s"", Toolbar, I, ""capture"", """"...","[thank, yahoo, ""s"", toolbar, i, ""captur"", """", ...","[Thanks, Yahoo, ""s"", Toolbar, I, ""capture"", """"...",Thanks to Yahoo's Toolbar I can now 'capture' ...,3,25.67
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",If anything Korea is a country of extremes. Ev...,en,en,"[If, anything, Korea, is, a, country, of, extr...","[If anything Korea is a country of extremes., ...",430,23,168,"[If, anything, Korea, country, extremes, ., Ev...","[if, anyth, korea, countri, extrem, ., everyth...","[If, anything, Korea, country, extreme, ., Eve...",If anything Korea is a country of extremes. Ev...,23,18.70
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Take a read of this news article from urlLink ...,en,en,"[Take, a, read, of, this, news, article, from,...","[""Take a read of this news article from urlLin...",430,11,158,"[Take, read, news, article, urlLink, JoongAng,...","[take, read, news, articl, urllink, joongang, ...","[Take, read, news, article, urlLink, JoongAng,...",Take a read of this news article from urlLink ...,11,39.09
7,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a lot looking fo...,en,en,"[I, surf, the, English, news, sites, a, lot, l...","[""I surf the English news sites a lot looking ...",183,7,64,"[I, surf, English, news, sites, lot, looking, ...","[i, surf, english, news, site, lot, look, tidb...","[I, surf, English, news, site, lot, looking, t...",I surf the English news sites a lot looking fo...,7,26.14
8,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",If you click on my profile you'll make a not-s...,en,en,"[If, you, click, on, my, profile, you, ""ll"", m...","[""If you click on my profile youll make a not-...",563,22,236,"[If, click, profile, ""ll"", make, not-so-startl...","[if, click, profil, ""ll"", make, not-so-startl,...","[If, click, profile, ""ll"", make, not-so-startl...",If you click on my profile you'll make a not-s...,22,25.59
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",Last night was pretty fun...mostly because of ...,en,en,"[Last, night, was, pretty, fun, ..., mostly, b...",[Last night was pretty fun...mostly because of...,658,23,230,"[Last, night, pretty, fun, ..., mostly, compan...","[last, night, pretti, fun, ..., mostli, compan...","[Last, night, pretty, fun, ..., mostly, compan...",Last night was pretty fun...mostly because of ...,23,28.61
10,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",There is so much that is different here from a...,en,en,"[There, is, so, much, that, is, different, her...","[""There is so much tha

In [141]:
df=df[df['topic']!='Maritime']

random  500 inhlate rausziehen für die jeweilige kategorie 

In [142]:
from sklearn.model_selection import train_test_split
X=df.drop(columns = ['gender','age','topic','sign'],axis=1)
y = df[['gender','age','topic','sign']]

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1,test_size=0.01)

In [144]:
df_train=pd.merge(X_train, y_train, left_index=True, right_index=True)
df_test=pd.merge(X_test, y_test, left_index=True, right_index=True)

In [145]:
df_train.to_csv('data/df_train.csv')
df_test.to_csv('data/df_test.csv')

In [146]:
df_lsit=pd.DataFrame(df_train['topic'].value_counts()).reset_index().rename(columns={"index": "topic", "topic": "num"})

In [147]:
df_lsit

,topic,num
0,indUnk,217471
1,Student,131955
2,Technology,36847
3,Arts,27453
4,Education,25218
5,Communications-Media,17130
6,Internet,13850
7,Non-Profit,12574
8,Engineering,9899
9,Law,7930


In [148]:
list_of_values= df_lsit[df_lsit['num']>500].topic.to_list()
list_of_values

['indUnk',
 'Student',
 'Technology',
 'Arts',
 'Education',
 'Communications-Media',
 'Internet',
 'Non-Profit',
 'Engineering',
 'Law',
 'Publishing',
 'Science',
 'Government',
 'Consulting',
 'Religion',
 'Fashion',
 'Marketing',
 'Advertising',
 'BusinessServices',
 'Banking',
 'Chemicals',
 'Accounting',
 'Telecommunications',
 'Museums-Libraries',
 'Military',
 'HumanResources',
 'Sports-Recreation',
 'RealEstate',
 'Transportation',
 'Biotech',
 'Manufacturing',
 'Tourism',
 'LawEnforcement-Security',
 'Architecture',
 'InvestmentBanking',
 'Agriculture',
 'Automotive',
 'Construction',
 'Environment']

In [149]:
len(df_train['topic'].unique())

39

In [150]:
df_train['topic'].unique()

array(['Student', 'indUnk', 'Arts', 'Publishing', 'Communications-Media',
       'Education', 'Technology', 'Consulting', 'LawEnforcement-Security',
       'Biotech', 'Government', 'Transportation', 'RealEstate',
       'Internet', 'Chemicals', 'Non-Profit', 'Telecommunications',
       'Museums-Libraries', 'Tourism', 'Engineering', 'InvestmentBanking',
       'Accounting', 'Science', 'BusinessServices', 'Military',
       'Religion', 'Law', 'Fashion', 'Construction', 'Environment',
       'Marketing', 'Manufacturing', 'Banking', 'Advertising',
       'Sports-Recreation', 'Automotive', 'HumanResources',
       'Architecture', 'Agriculture'], dtype=object)

In [151]:
def loop(df_2,col,anzahl=500):
    liste_index=[]
    topic_list=df_2[col].unique()
    for element in tqdm(topic_list):
        liste=df_2[df_2['topic']==element].sample(n=anzahl, replace = False,random_state = 2).index.tolist()
        liste_index=liste_index+liste
    return liste_index
    

In [152]:
df_500_liste=loop(df_train,'topic')

100%|██████████| 39/39 [00:06<00:00,  6.13it/s]


In [153]:
len(df_500_liste)

19500

In [154]:
df_train.loc[df_500_liste].topic.value_counts()

Student                    500
Environment                500
Science                    500
BusinessServices           500
Military                   500
Religion                   500
Law                        500
Fashion                    500
Construction               500
Marketing                  500
InvestmentBanking          500
Manufacturing              500
Banking                    500
Advertising                500
Sports-Recreation          500
Automotive                 500
HumanResources             500
Architecture               500
Accounting                 500
Engineering                500
indUnk                     500
Biotech                    500
Arts                       500
Publishing                 500
Communications-Media       500
Education                  500
Technology                 500
Consulting                 500
LawEnforcement-Security    500
Government                 500
Tourism                    500
Transportation             500
RealEsta

In [155]:
df_500=df_train.loc[df_500_liste]

In [156]:
df_500.to_csv('data/train_500.csv')

In [162]:
df_500['text'][445164]

'This blog is being posted due to the fact that little development has happened recently regarding the last day of school uniform swap. If we all plan to switch uniforms with a certain individual we must somehow organize something soon. Also what do we consider the last day of school I think if everyone participates this will be a success. We must plan this soon and pray that there is not a drop-the-dress-code-day on the last day of school.'

In [158]:
df_500.head(5)

,id,date,text,lang,language_2,word_tokenize,sent_tokenize,count_word,count_sent,word_tokenize_num_of_stopwords,word_tokenize_without_stopwords,word_tokenize_without_stopwords_port,msg_lemmatized,cleanLinks,laenge_saetze,word_per_sent_mean,gender,age,topic,sign
445164,2970791,"25,May,2004",This blog is being posted due to the fact that...,en,en,"[This, blog, is, being, posted, due, to, the, ...",[This blog is being posted due to the fact tha...,83,4,34,"[This, blog, posted, due, fact, little, develo...","[thi, blog, post, due, fact, littl, develop, h...","[This, blog, posted, due, fact, little, develo...",This blog is being posted due to the fact that...,4,20.75,male,14,Student,Sagittarius
293091,3931851,"02,agosto,2004",So I have a big fucking interview tomorrow for...,en,en,"[So, I, have, a, big, fucking, interview, tomo...",[So I have a big fucking interview tomorrow fo...,88,5,31,"[So, I, big, fucking, interview, tomorrow, new...","[so, i, big, fuck, interview, tomorrow, new, s...","[So, I, big, fucking, interview, tomorrow, new...",So I have a big fucking interview tomorrow for...,5,17.60,male,15,Student,Pisces
594697,3481650,"07,July,2004",I was reminded just now of the time Ashley and...,en,en,"[I, was, reminded, just, now, of, the, time, A...",[I was reminded just now of the time Ashley an...,574,32,234,"[I, reminded, time, Ashley, I, drove, Kemah, w...","[i, remind, time, ashley, i, drove, kemah, win...","[I, reminded, time, Ashley, I, drove, Kemah, w...",I was reminded just now of the time Ashley and...,32,17.94,female,17,Student,Gemini
3764,4187211,"20,August,2004",I was checking up on my cousin Dylan and Fanni...,en,en,"[I, was, checking, up, on, my, cousin, Dylan, ...","[""I was checking up on my cousin Dylan and Fan...",84,3,32,"[I, checking, cousin, Dylan, Fannie, ""s"", wedd...","[i, check, cousin, dylan, fanni, ""s"", wed, sit...","[I, checking, cousin, Dylan, Fannie, ""s"", wedd...",I was checking up on my cousin Dylan and Fanni...,3,28.00,female,23,Student,Taurus
162953,3686696,"24,June,2004",for the NME interview click urlLink part 1 and...,en,en,"[for, the, NME, interview, click, urlLink, par...",[for the NME interview click urlLink part 1 an...,12,1,3,"[NME, interview, click, urlLink, part, 1, urlL...","[nme, interview, click, urllink, part, 1, urll...","[NME, interview, click, urlLink, part, 1, urlL...",for the NME interview click urlLink part 1 and...,1,12.00,female,23,Student,Aquarius
